In [37]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O sarcasm.json

'wget' is not recognized as an internal or external command,
operable program or batch file.



# Instalacja pakietów

```bash
pip install --upgrade pip
pip install "tensorflow"
```

Źródło: https://goo.gle/nlp-z2h



Źródło: https://goo.gle/nlp-z2h

In [38]:
import json

In [39]:
with open("sarcasm.json", "r") as f:
    datastore = json.load(f)
    

In [40]:
sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
seqences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(seqences, padding="post")
print(padded[0])
print(padded.shape)
print()

[  308 15115   679  3337  2298    48   382  2576 15116     6  2577  8434
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
(26709, 40)



In [43]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [44]:
training_size = int(padded.shape[0] * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [45]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [46]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_5   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_10 (Dense)            (None, 24)                408       
                                                                 
 dense_11 (Dense)            (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [47]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
668/668 - 5s - loss: 0.6695 - accuracy: 0.5855 - val_loss: 0.5986 - val_accuracy: 0.7398 - 5s/epoch - 8ms/step
Epoch 2/30
668/668 - 3s - loss: 0.4524 - accuracy: 0.8268 - val_loss: 0.4110 - val_accuracy: 0.8044 - 3s/epoch - 4ms/step
Epoch 3/30
668/668 - 3s - loss: 0.3262 - accuracy: 0.8704 - val_loss: 0.3547 - val_accuracy: 0.8525 - 3s/epoch - 5ms/step
Epoch 4/30
668/668 - 3s - loss: 0.2742 - accuracy: 0.8919 - val_loss: 0.3415 - val_accuracy: 0.8562 - 3s/epoch - 5ms/step
Epoch 5/30
668/668 - 3s - loss: 0.2386 - accuracy: 0.9081 - val_loss: 0.3386 - val_accuracy: 0.8547 - 3s/epoch - 4ms/step
Epoch 6/30
668/668 - 3s - loss: 0.2125 - accuracy: 0.9188 - val_loss: 0.3461 - val_accuracy: 0.8540 - 3s/epoch - 4ms/step
Epoch 7/30
668/668 - 3s - loss: 0.1923 - accuracy: 0.9277 - val_loss: 0.3496 - val_accuracy: 0.8534 - 3s/epoch - 4ms/step
Epoch 8/30
668/668 - 3s - loss: 0.1723 - accuracy: 0.9362 - val_loss: 0.3690 - val_accuracy: 0.8487 - 3s/epoch - 4ms/step
Epoch 9/30
668/668 - 3s 

In [48]:
sentence = [
    'granny starting to fear spiders in the garden might be real',
    'the weather today is bright and sunny'
]
seqences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(seqences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

1/1 [==============================] - 0s 163ms/step
[[9.6163088e-01]
 [1.8236817e-04]]
